In [15]:
!pip install datasets pandas google-genai numpy tiktoken nltk


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [97]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset
import pandas as pd

from tqdm.auto import tqdm
import pickle

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import html
import nltk
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
    nltk.download('wordnet')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
ds = load_dataset("OpenPipe/hacker-news") # streaming=True
df = pd.DataFrame(ds['train'][-1_000_000:-1])

stories = df[df.type == 'story']
comments = df[df.type == 'comment']

df.tail(3)

,id,type,by,time,title,text,url,score,parent,top_level_parent,descendants,kids,deleted,dead
999996,41813381,comment,marcosdumay,2024-10-11 20:26:21,None,Add forced sedentarism into that set.<p>This i...,None,NaN,41812891.0,41811263,NaN,None,None,None
999997,41813382,story,turkeynecks,2024-10-11 20:26:24,Mini DOOM-like FPS in BooBoo programming language,None,https://www.indiedb.com/games/doomed1,1.0,NaN,41813382,0.0,None,None,None
999998,41813383,comment,davio,2024-10-11 20:26:28,None,hims sells the generic version for a fraction ...,None,NaN,41813102.0,41811263,NaN,None,None,None


## Clean comments

In [103]:
bad_comment = ['[flagged]', '[dead]', 'Thanks!', 'Thank you!', 'Yes.', 'No.', 'Yes', 'No', 'Thanks', 'Thank you']
comments['low_quality'] = comments.text.map(lambda x: x in bad_comment) | comments.text.isna()
comments = comments[~comments.low_quality]

C:\Users\David\AppData\Local\Temp\ipykernel_21508\678916102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['low_quality'] = comments.text.map(lambda x: x in bad_comment) | comments.text.isna()


In [104]:
comments['text'] = comments.text.map(lambda x: html.unescape(x))

In [105]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """Lowercase, remove punctuation, remove stop words, and lemmatize text."""
    if not isinstance(text, str):
        return "" # Handle non-string values

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
    
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

In [108]:
comments['clean_text'] = comments['text'].apply(preprocess_text)

In [107]:
len(comments)

822993

## Get Queries

In [109]:
from keywords import HnKeywords
keywords = HnKeywords.as_dict()
keywords.keys()

dict_keys(['startup_founder_issues', 'specific_tools_methodologies', 'critiques_of_market_research', 'jobs_to_be_done', 'market_research_terms', 'overlapping_terms', 'customer_interviews_terms', 'product_validation_terms'])

In [110]:
search_terms = dict()
for label, phrases in keywords.items():
    clean = [preprocess_text(phrase) for phrase in phrases]
    search_terms[label] = list(set(clean))
search_terms

{'startup_founder_issues': ['launching startup',
  'idea validation',
  'getting first customer',
  'go market strategy',
  'startup pain point',
  'building mvp',
  'startup mistake'],
 'specific_tools_methodologies': ['google form',
  'typeform',
  'surveymonkey',
  'dovetail',
  'qualitative analysis software',
  'nvivo'],
 'critiques_of_market_research': ['useless customer interview',
  'bad user research',
  'outdated market research',
  'time consuming research',
  'market research bias',
  'biased user feedback',
  'flaw market analysis',
  'expensive market research',
  'market research broken',
  'ineffective feedback',
  'problem market research'],
 'jobs_to_be_done': ['job customer hire product',
  'job done case study',
  'understanding job',
  'job done framework',
  'jtbd implementation',
  'jtbd example'],
 'market_research_terms': ['market trend',
  'market segmentation strategy',
  'competitive benchmarking',
  'understanding market dynamic',
  'market research report'

## Search and save

In [126]:
def keyword_search(corpus, queries, pbar=None):
    """Returns a boolean mask for rows that contain any of the given queries."""
    if not queries:
        if pbar is not None:
            pbar.update(0)
        return pd.Series(False, index=corpus.index)
    
    pattern = r'\b(?:' + '|'.join(re.escape(q) for q in queries) + r')\b'
    compiled_pattern = re.compile(pattern, re.IGNORECASE)
    
    mask = corpus.str.contains(compiled_pattern, na=False)
    if pbar is not None:
        pbar.update(len(queries))
    return mask


In [127]:
total_query_count = sum(len(terms) for terms in search_terms.values())
pbar = tqdm(total=total_query_count, unit='queries')

comments['labels'] = [[] for _ in range(len(comments))]

# 3. For each category, find rows that match the category queries
for category, queries in search_terms.items():
    mask = keyword_search(comments['clean_text'], queries, pbar=pbar)
    # Append the category label to each matching row's 'labels' list
    comments.loc[mask, 'labels'] = comments.loc[mask, 'labels'].apply(
        lambda current_list: current_list + [category]
    )
pbar.close()

100%|██████████| 185/185 [01:22<00:00,  2.24queries/s]


In [ ]:
# Pickles the comment IDs and their matches
timestamp = pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
comments['labels_str'] = comments['labels'].apply(lambda labels: '|'.join(labels))
comments_with_labels = comments[comments['labels_str'] != '']
comments_with_labels[['id', 'labels_str']].to_csv(
    f'out/comments_with_labels_{timestamp}.csv', index=False
)

In [128]:
comments.labels.explode().value_counts()

labels
overlapping_terms               1005
product_validation_terms         267
market_research_terms            242
customer_interviews_terms        242
specific_tools_methodologies      76
startup_founder_issues            26
jobs_to_be_done                    4
Name: count, dtype: int64